# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучим модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

Построим модель со значением метрики качества *F1* не меньше 0.75. 

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
#Импортируем необходимые библиотеки
import pandas as pd
import numpy as np

import pymorphy2
from pymystem3 import Mystem
import re

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score
from sklearn.utils import shuffle

import nltk
from nltk.corpus import stopwords as nltk_stopwords
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
#Откроем файл и изучим первые 10 строк
df = pd.read_csv('/datasets/toxic_comments.csv')
df.head(10)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
5,"""\n\nCongratulations from me as well, use the ...",0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,Your vandalism to the Matt Shirvington article...,0
8,Sorry if the word 'nonsense' was offensive to ...,0
9,alignment on this subject and which are contra...,0


In [3]:
#Изучим данные
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [4]:
#Проверим количество дубликатов
df.duplicated().sum()

0

In [5]:
#Проверим количество значений столбца 'toxic'
df['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

In [6]:
#Посмотрим отношение значений столбца 'toxic'
df['toxic'].value_counts()[1] / df['toxic'].value_counts()[0]

0.1131876717871444

In [7]:
#Напишем функцию лемматизации с регулярными выражениями
m = Mystem() 

def lemmatize(text):
    text = text.lower()
    corpus = ''.join(m.lemmatize(text))
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    text = ' '.join(text.split()) 
    return text

In [8]:
#Применим функцию к новом столбцу 'lemm_text' и удалим старый столбец 'text'
df['lemm_text'] = df['text'].apply(lemmatize)
df = df.drop('text', axis=1)

In [9]:
#Проверим изменения
df.head(10)

,toxic,lemm_text
0,0,explanation why the edits made under my userna...
1,0,d aww he matches this background colour i m se...
2,0,hey man i m really not trying to edit war it s...
3,0,more i can t make any real suggestions on impr...
4,0,you sir are my hero any chance you remember wh...
5,0,congratulations from me as well use the tools ...
6,1,cocksucker before you piss around on my work
7,0,your vandalism to the matt shirvington article...
8,0,sorry if the word nonsense was offensive to yo...
9,0,alignment on this subject and which are contra...


### Вывод

С данными всё в порядке, дубликатов нет. Была проведена лемматизация и очистка текста с помощью регулярных выражений. В значениях столбца 'toxic' присутствует дисбаланс (значения '1' встречаются реже значения '0' примерно в 10 раз), что может плохо сказаться на обучении моделей.

## Обучение

In [10]:
#Выделим признаки и разделим данные на тренировочную, валидационную и тестовую выборки
X = df.drop('toxic', axis=1)
y = df['toxic']
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.4, random_state=12345)
X_valid, X_test, y_valid, y_test = train_test_split(X_valid, y_valid, test_size=0.5, random_state=12345)

In [11]:
#Проверим размеры выборок
print('X_train:', X_train.shape)
print('X_valid:', X_valid.shape)
print('X_test:', X_test.shape)

X_train: (95742, 1)
X_valid: (31914, 1)
X_test: (31915, 1)


In [12]:
#Найдём величину TF-IDF и применим её к нашим выборкам
count_tf_idf = TfidfVectorizer(stop_words=stopwords)
X_train = count_tf_idf.fit_transform(X_train['lemm_text'].values.astype('U'))
X_valid = count_tf_idf.transform(X_valid['lemm_text'].values.astype('U'))
X_test = count_tf_idf.transform(X_test['lemm_text'].values.astype('U'))
print('X_train:', X_train.shape)
print('X_valid:', X_valid.shape)
print('X_test:', X_test.shape)

X_train: (95742, 125610)
X_valid: (31914, 125610)
X_test: (31915, 125610)


In [13]:
#Применим downsampling: напишем функцию для уменьшения дисбаланса классов
def downsample(features, target, fraction):
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    target_zeros_downsample = target_zeros.sample(frac=fraction, random_state=12345)
    target_downsampled = pd.concat([target_zeros_downsample, target_ones])
    features_downsampled = df.iloc[target_downsampled.index]
    features_downsampled, target_downsampled = shuffle(features_downsampled, target_downsampled, random_state=12345)
    features_downsampled = count_tf_idf.transform(features_downsampled['lemm_text'].values.astype('U'))
    return features_downsampled, target_downsampled

In [14]:
#Применим функцию, но возьмём fraction = 0.5, поскольку соотношение положительных и отрицательных комментариев не должно 
#быть одинаковым
X_downsampled_train, y_downsampled_train = downsample(X_train, y_train, 0.5)

In [15]:
#Для LogisticRegression найдём лучшие параметры, которые будут использованы на тестовой выборке и обучим модель 
parameters_log_reg = {'class_weight': ['balanced', None], 'random_state': [12345],
                      'C':[0.1, 1, 5], 'solver':['newton-cg', 'lbfgs', 'liblinear']}                   
model_log_reg = LogisticRegression()
grid_log_reg = GridSearchCV(estimator=model_log_reg, scoring='f1', param_grid=parameters_log_reg, cv=5)
grid_log_reg.fit(X_downsampled_train, y_downsampled_train)
predicted_y = grid_log_reg.predict(X_valid)
log_reg_best_params = grid_log_reg.best_params_
print(log_reg_best_params)

{'C': 5, 'class_weight': 'balanced', 'random_state': 12345, 'solver': 'newton-cg'}


In [16]:
#Для SGDClassifier найдём лучшие параметры, которые будут использованы на тестовой выборке и обучим модель 
parameters_SGDC = {'loss': ['hinge', 'log', 'modified_huber', 'huber'],
                   'n_jobs': [-1, None, 1], 'shuffle': [True, False], 'random_state': [12345], 
                   'learning_rate': ['constant', 'optimal', 'invscaling', 'adaptive'],
                   'eta0': [0.01, 0.05, 0.1, 0.25, 0.5]}
model_SGDC = SGDClassifier()
grid_SGDC = GridSearchCV(estimator=model_SGDC, param_grid=parameters_SGDC, scoring='f1', cv=5)
grid_SGDC.fit(X_downsampled_train, y_downsampled_train)
predicted_y = grid_SGDC.predict(X_valid)
SGDC_best_params = grid_SGDC.best_params_
print(SGDC_best_params)

{'eta0': 0.1, 'learning_rate': 'constant', 'loss': 'modified_huber', 'n_jobs': -1, 'random_state': 12345, 'shuffle': True}


In [17]:
#Для RandomForestClassifier найдём лучшие параметры, которые будут использованы на тестовой выборке и обучим модель 
parameters_RFC = {'n_estimators': [10, 50, 100], 'max_depth': [10, 30], 'random_state': [12345], 
                  'n_jobs': [-1, None, 1], 'criterion': ['gini', 'entropy', 'log_loss'], 
                  'class_weight': ['balanced', None]}
model_RFC = RandomForestClassifier()
grid_RFC = GridSearchCV(estimator=model_RFC, param_grid=parameters_RFC, scoring='f1', cv=5)
grid_RFC.fit(X_downsampled_train, y_downsampled_train)
predicted_y = grid_RFC.predict(X_valid)
RFC_best_params = grid_RFC.best_params_
print(RFC_best_params)

{'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 30, 'n_estimators': 100, 'n_jobs': -1, 'random_state': 12345}


In [18]:
#Для DecisionTreeClassifier найдём лучшие параметры, которые будут использованы на тестовой выборке и обучим модель 
parameters_DTC = {'splitter': ['best', 'random'], 'max_depth': [10, 30, 50], 'random_state': [12345], 
                  'criterion': ['gini', 'entropy', 'log_loss'], 'class_weight': ['balanced', None]}
model_DTC = DecisionTreeClassifier()
grid_DTC = GridSearchCV(estimator=model_DTC, param_grid=parameters_DTC, scoring='f1', cv=5)
grid_DTC.fit(X_downsampled_train, y_downsampled_train)
predicted_y = grid_DTC.predict(X_valid)
DTC_best_params = grid_DTC.best_params_
print(DTC_best_params)

{'class_weight': None, 'criterion': 'gini', 'max_depth': 50, 'random_state': 12345, 'splitter': 'best'}


### Вывод

Для обучения моделей было сделано 3 выборки:

Размер выборки 'train': 60% (95742);

Размер выборки 'valid': 20% (31914);

Размер выборки 'test': 20% (31915);

К признакам был применён метод TfidfVectorizer. Дисбаланс классов был изменён с помощью downsampling.

Для моделей были подобраны лучшие параметры с помощью GridSearchCV(), который будут использоваться для нахождения F1 для тестовой выборки.

**Лучшие параметры для моделей:**

LogisticRegression best parameters: C=5, class_weight='balanced', random_state=12345, solver='newton-cg'

SGDClassifier best parameters: eta0=0.1, learning_rate='constant', loss='modified_huber', n_jobs=-1, random_state=12345, shuffle=True

RandomForestClassifier best parameters: class_weight='balanced', criterion='entropy', max_depth=30, n_estimators=100, n_jobs=-1, random_state=12345

DecisionTreeClassifier best parameters: class_weight=None, criterion='gini', max_depth=50, random_state=12345, splitter='best'

## Выводы

In [19]:
#Найдём F1 для DecisionTreeClassifier для тестовой выборки
model_DTC = DecisionTreeClassifier(**DTC_best_params)
model_DTC.fit(X_downsampled_train, y_downsampled_train)
DTC_predict = model_DTC.predict(X_test)
print('F1 for DecisionTreeClassifier test:', f1_score(y_test, DTC_predict))

F1 for DecisionTreeClassifier test: 0.6991256643236756


In [20]:
#Найдём F1 для RandomForestClassifier для тестовой выборки
model_RFC.set_params(**RFC_best_params)
model_RFC.fit(X_downsampled_train, y_downsampled_train)
RFC_predict = model_RFC.predict(X_test)
print('F1 for RandomForestClassifier test:', f1_score(y_test, RFC_predict))

F1 for RandomForestClassifier test: 0.4320234317793507


In [21]:
#Найдём F1 для SGDClassifier для тестовой выборки
model_SGDC.set_params(**SGDC_best_params)
model_SGDC.fit(X_downsampled_train, y_downsampled_train)
SGDC_predict = model_SGDC.predict(X_test)
print('F1 for SGDClassifier test:', f1_score(y_test, SGDC_predict))

F1 for SGDClassifier test: 0.7731149778232685


In [22]:
#Найдём F1 для LogisticRegression для тестовой выборки
model_log_reg.set_params(**log_reg_best_params)
model_log_reg.fit(X_downsampled_train, y_downsampled_train)
log_reg_predict = model_log_reg.predict(X_test)
print('F1 for LogisticRegression test:', f1_score(y_test, log_reg_predict))

F1 for LogisticRegression test: 0.7496883224823383


In [25]:
#Выведем итог на экран
data = {'F1':[f1_score(y_test, log_reg_predict),
              f1_score(y_test, SGDC_predict),
              f1_score(y_test, RFC_predict),
              f1_score(y_test, DTC_predict)]}
index = ['LogisticRegression', 'SGDClassifier', 'RandomForestClassifier', 'DecisionTreeClassifier']
pd.DataFrame(data=data, index=index)

,F1
LogisticRegression,0.749688
SGDClassifier,0.773115
RandomForestClassifier,0.432023
DecisionTreeClassifier,0.699126


### Вывод

На тестовых вариантах мы имеем:
- F1 for DecisionTreeClassifier test: 0.6991256643236756
- F1 for RandomForestClassifier test: 0.4320234317793507
- F1 for SGDClassifier test: 0.7731149778232685
- F1 for LogisticRegression test: 0.7496883224823383

Можно считать, что 2 из 4 моделей имеют метрику F1 > 0.75 (LogisticRegression очень близка к этому). SDGClassifier имеет наилучшую метрику, а RandomForestClassifier имеет наихудшую метрику.

## Общий вывод

Можно сказать, что самое долгое обучение протекало у моделей RandomForestClassifier и у LogisticRegression, причём первая имеет наихудшую метрику F1. 

Можно попробовать добавить больше гиперпараметров во все модели и найти новые наилучшие параметры, но тогда на их обучение может потребоваться в разы больше времени.

**ИТОГ: Самая лучшая модель для обучения - модель SGDClassifier с параметрами:**

eta0=0.1, learning_rate='constant', loss='modified_huber', n_jobs=-1, random_state=12345, shuffle=True
